# Add Plugins Step by Step - using PlugnPlai and LangChain

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/edreisMD/plugnplai/blob/main/examples/plugins_step_by_step.ipynb)

The goal of this example is to go through all the steps to add plugins to LLMs
1. Get plugins of certain categories from [plugnplai.com](https://plugnplai.com)
2. Load plugins manifest and specifications
3. Parse specifications and generate a prompt with the descriptions
4. Use [LangChain]() to call the LLM
5. Parse the LLM response, looking for the `[API]` pattern defined on `plugins.prompt`
6. Call the plugin using `plugins.call()`
7. Use LangChain again to ask the LLM a final response using the new data

## Install

In [1]:
pip install plugnplai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.3/756.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.6 MB/s eta 0:00:00


## Get the plugins

We want to install at maximum three plugins, in order to fit the description on the context length

Lets find one plugin for each category:
1. travel
2. shopping 
3. weather

We can use PlugnPlai categories (see [API reference](https://plugnplai.github.io/))

In [1]:
import plugnplai as pl

# Get working plugins - only tested plugins (in progress)
urlsTravel = pl.get_plugins(category='travel')
print(f'Travel plugins: {urlsTravel}')

urlsShopping = pl.get_plugins(category='shopping')
print(f'Shopping plugins: {urlsShopping}')

urlsLan = pl.get_plugins(category='language')
print(f'Weather plugins: {urlsLan}')

Travel plugins: ['https://llmsearch.endpoint.getyourguide.com', 'https://gogaffl.com', 'https://trip.com', 'https://api.yelp.com', 'https://gps-telecom.com']
Shopping plugins: ['https://gptshop.bohita.com', 'https://lexi-shopping-assistant-chatgpt-plugin.iamnazzty.repl.co', 'https://pricerunner.com', 'https://server.shop.app', 'https://klarna.com']
Weather plugins: ['https://api.speak.com']


In [3]:
# Lets pick one of each list and add to our url list

urls = []

# Trip (list index 1)
urls.append([plugin for plugin in urlsTravel if 'trip' in plugin][0])

# Klarna (list index 2)
urls.append([plugin for plugin in urlsShopping if 'klarna' in plugin][0])

# Speak (list index 0)
urls.append([plugin for plugin in urlsLan if 'speak' in plugin][0])

print(f'Our chosen Plugins: {urls}')

Our chosen Plugins: ['https://trip.com', 'https://klarna.com', 'https://api.speak.com']


## Load and activate the plugins

In [4]:
from plugnplai import Plugins

plugins = Plugins.install_and_activate(urls)

## Print the default prompt for the active plugins

In [5]:
print(plugins.prompt)


# SYSTEM MESSAGE
You are a large language model trained to assist humans.
Knowledge Cutoff: 2021-09
Current date: 2023-05-13
Below is a list of available APIs that you can utilize to fulfill user requests. 
When using an API, please follow the specified format to make the API call. 
If possible, avoid asking follow-up questions and aim to complete the task with the information provided by the user.

To make an API call, use the following format:

[API]namespace.operationId[/API]
[PARAMS]{ 
    "parameter_name": "parameter_value",
    ...
}[/PARAMS]

For example, to call an API operation with the operation ID "productsUsingGET" in the "KlarnaProducts" namespace, 
and provide the required parameters "q" and "size", the format would be as follows:

[API]KlarnaProducts.productsUsingGET[/API]
[PARAMS]{
    "q": "t-shirt", 
    "size": 3
}[/PARAMS]

Please ensure that you use the correct namespace and operation ID, and provide the necessary parameters for each API call. 
After requesting th

## Lets look at the length of the prompt

Get the number of tokens of the prompt by just calling 'plugins.tokens' 

In [6]:
print(plugins.tokens)

1212


## Call the LLM using LangChain

In [8]:
# You will need to first define your API key
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_KEY"

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage

#### Uncomment or modify the message to test different plugins

In [20]:
# Test Klarna Plugin
# HUMAN_MESSAGE = "I want to buy a rolling stones t-shirt"

# Test Trip Plugin
HUMAN_MESSAGE = "I need a hotel in Paris between Dec.3-8"

# Test Speak Plugin
# HUMAN_MESSAGE = "How to say I love you in Portuguese?"

#### Call LLM

In [21]:
chat = ChatOpenAI(temperature=0, model="gpt-4")

messages = [
    SystemMessage(content=plugins.prompt),
    HumanMessage(content=HUMAN_MESSAGE)
]

res = chat(messages)

llm_first_response = res.content

print(llm_first_response)

[API]Trip.search_hotel[/API]
[PARAMS]{
    "cityName": "Paris",
    "topHotel": 5,
    "locale": "en",
    "checkIn": "2023-12-03",
    "checkOut": "2023-12-08"
}[/PARAMS]


## Parse the LLM response

In [23]:
# import the parser function
from plugnplai import parse_llm_response

# Parse the LLM response importing '
call_dict = parse_llm_response(llm_first_response)
print(call_dict)

{'plugin_name': 'Trip', 'operation_id': 'search_hotel', 'parameters': {'cityName': 'Paris', 'topHotel': 5, 'locale': 'en', 'checkIn': '2023-12-03', 'checkOut': '2023-12-08'}}


## Call Plugin

In [24]:
r = plugins.call_api(plugin_name = call_dict['plugin_name'], 
                    operation_id = call_dict['operation_id'], 
                    parameters = call_dict['parameters']
                    )

api_response = r.json()
r.json()

{'hotelList': [{'hotelName': 'Le Tsuba Hotel',
   'hotelDescription': '',
   'hotelPrice': 295.0,
   'hotelCurrency': 'USD',
   'hotelLink': 'https://us.trip.com/hotels/detail/?cityId=192&hotelId=6597288&checkin=2023-12-03&checkout=2023-12-08&curr=USD',
   'hotelAddress': '45 Rue des Acacias',
   'openYear': '2017',
   'renovationYear': '2017',
   'score': '4.6/5.0',
   'numberOfReviews': 36,
   'numberOfFavorites': 'Saved by 4115 person',
   'star': 4,
   'hotelFeatureTagList': ['桑拿', '健身室']},
  {'hotelName': 'Pullman Paris Centre - Bercy',
   'hotelDescription': '',
   'hotelPrice': 262.0,
   'hotelCurrency': 'USD',
   'hotelLink': 'https://us.trip.com/hotels/detail/?cityId=192&hotelId=2107175&checkin=2023-12-03&checkout=2023-12-08&curr=USD',
   'hotelAddress': '1 Rue de Libourne',
   'openYear': '2000',
   'renovationYear': '2007',
   'score': '4.5/5.0',
   'numberOfReviews': 42,
   'numberOfFavorites': 'Saved by 3731 person',
   'star': 4,
   'hotelFeatureTagList': ['泳池', '儿童乐园']},

## LLM responds using the API data

In [25]:
api_return_prompt = f"""
Assistant is a large language model with access to plugins.

Assistant called a plugin in response to this human message:
# HUMAN MESSAGE
{HUMAN_MESSAGE}

# API REQUEST SUMMARY
{llm_first_response}

# API RESPONSE
{api_response}
"""

# Install the plugins ewith the original template
plugins = Plugins.install_and_activate(urls)

chat = ChatOpenAI(temperature=0, model="gpt-4")
# chat = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

messages = [
    SystemMessage(content=api_return_prompt),
    HumanMessage(content="HUMAN_MESSAGE")
]

res = chat(messages)

print(res.content)

I found 5 hotels in Paris for your stay between December 3rd and December 8th. Here are the top options:

1. [Le Tsuba Hotel](https://us.trip.com/hotels/detail/?cityId=192&hotelId=6597288&checkin=2023-12-03&checkout=2023-12-08&curr=USD)
   - Address: 45 Rue des Acacias
   - Price: $295 per night
   - Rating: 4.6/5.0
   - Features: Sauna, Fitness Room

2. [Pullman Paris Centre - Bercy](https://us.trip.com/hotels/detail/?cityId=192&hotelId=2107175&checkin=2023-12-03&checkout=2023-12-08&curr=USD)
   - Address: 1 Rue de Libourne
   - Price: $262 per night
   - Rating: 4.5/5.0
   - Features: Swimming Pool, Children's Playground

3. [Pullman Paris Tour Eiffel](https://us.trip.com/hotels/detail/?cityId=192&hotelId=2081163&checkin=2023-12-03&checkout=2023-12-08&curr=USD)
   - Address: 18 Avenue De Suffren, 22 Rue Jean Rey Entrée Au
   - Price: $298 per night
   - Rating: 4.2/5.0
   - Features: Fitness Room, Multi-function Hall

4. [Hotel de Castiglione Paris](https://us.trip.com/hotels/detail/

## Lets copy the response to a markdown cell:

I found 5 hotels in Paris for your stay between December 3rd and December 8th. Here are the top options:

1. [Le Tsuba Hotel](https://us.trip.com/hotels/detail/?cityId=192&hotelId=6597288&checkin=2023-12-03&checkout=2023-12-08&curr=USD)
   - Address: 45 Rue des Acacias
   - Price: $295 per night
   - Rating: 4.6/5.0
   - Features: Sauna, Fitness Room

2. [Pullman Paris Centre - Bercy](https://us.trip.com/hotels/detail/?cityId=192&hotelId=2107175&checkin=2023-12-03&checkout=2023-12-08&curr=USD)
   - Address: 1 Rue de Libourne
   - Price: $262 per night
   - Rating: 4.5/5.0
   - Features: Swimming Pool, Children's Playground

3. [Pullman Paris Tour Eiffel](https://us.trip.com/hotels/detail/?cityId=192&hotelId=2081163&checkin=2023-12-03&checkout=2023-12-08&curr=USD)
   - Address: 18 Avenue De Suffren, 22 Rue Jean Rey Entrée Au
   - Price: $298 per night
   - Rating: 4.2/5.0
   - Features: Fitness Room, Multi-function Hall

4. [Hotel de Castiglione Paris](https://us.trip.com/hotels/detail/?cityId=192&hotelId=2157992&checkin=2023-12-03&checkout=2023-12-08&curr=USD)
   - Address: 38-40 Rue du Faubourg Saint-Honoré
   - Price: $221 per night
   - Rating: 3.9/5.0
   - Features: Tea Room, Conference Hall

5. [Hotel de Crillon A Rosewood Hotel](https://us.trip.com/hotels/detail/?cityId=192&hotelId=1619850&checkin=2023-12-03&checkout=2023-12-08&curr=USD)
   - Address: 10 Pl. de la Concorde
   - Price: $1673 per night
   - Rating: 4.7/5.0
   - Features: Sunbathing Area, Sauna

Please note that prices and availability are subject to change. Make sure to book your preferred hotel as soon as possible to secure your reservation.